In [1]:
import torch
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
TB_writer = SummaryWriter()
from GAN_model.resnetGAN import *
from GAN_model.ProteoGAN import *
from utils.dataloader import *
from utils.torch_utils import *
from utils.metrics import *

In [2]:
args_run_name = 'ProteoGAN_HIS7_MSA'
# args; TODO: move these to parser for script
args_batch_size=64; args_seed=42; args_eval_n_seq = 500
args_epoch = 300
# GAN specific param
args_z_dim = 128; args_dim = 256
# optimizer specific param
args_lr_Gen = 1e-4; args_lr_Disc = 8e-5; args_loss = 'hinge'
# train discriminator more initially
# according to https://livebook.manning.com/book/gans-in-action/chapter-5/185
args_disc_iters_init = 2; args_disc_iters = 1; args_disc_iters_init_epoch = 50
# make dir
os.makedirs('checkpoints/'+args_run_name, exist_ok=True)
os.makedirs('out/'+args_run_name+'/plot_out/', exist_ok=True)
os.makedirs('out/'+args_run_name+'/seq_out/', exist_ok=True)

In [3]:
seed_everything(args_seed)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
# keep dataset in cpu
MSA_data = MSA_dataset('HIS7_data/his7_MSA.fasta', 
    torch.device('cpu'), pseudo_MSA=False) 

100%|██████████| 14747/14747 [00:12<00:00, 1141.19it/s]


### Creating data sets

In [4]:
# creating data set with train/val split
train_len = int(MSA_data.length*0.85-MSA_data.length)
val_len = MSA_data.length - train_len
train_data, val_data = torch.utils.data.random_split(
    MSA_data, [train_len, val_len], 
    generator=torch.Generator().manual_seed(args_seed))
# decoding train and validation sequences for evaluation
val_seqs = [decode_one_seq(torch.argmax(data, dim=-1).detach().cpu()) 
            for data in val_data]
tr_seqs = [decode_one_seq(torch.argmax(data, dim=-1).detach().cpu()) 
           for data in train_data]
# creating dataloader for training data
loader = torch.utils.data.DataLoader(
    train_data, batch_size=args_batch_size, shuffle=True,
    num_workers=1, pin_memory=True)
# fixed parameters
n_chars = 21; seq_len = len(val_seqs[0])

### Evaluating positive and negative control sequences to obtain baseline metrics
Positive: sample of real sequences and simulates a perfect model

Negative: a sample that simulates the worst possible model for each metric: \
    (constant sequence for MMD, repeated sequences for diversity measures).


In [5]:
pos_ctrl_seqs = np.random.choice(tr_seqs, size=len(val_seqs), replace=False)
# constant sequences for MMD
neg_ctrl_seqs_MMD = []
for _ in range(len(val_seqs)):
    rand_aa_idx = np.random.randint(0, high=19)
    rand_seq = decode_one_seq(
        np.random.randint(rand_aa_idx, high=rand_aa_idx+2, size=seq_len))
    neg_ctrl_seqs_MMD.append(rand_seq)
# repeated sequences for diversity measures
rand_seq = decode_one_seq(np.random.randint(0, high=21, size=seq_len))
neg_ctrl_seqs_diversity = [rand_seq for _ in range(len(val_seqs))]
# report baseline
MMD_pos = mmd(seq1=pos_ctrl_seqs, seq2=val_seqs)
MMD_neg = mmd(seq1=neg_ctrl_seqs_MMD, seq2=val_seqs)
Entropy_pos = abs(entropy(seq1=pos_ctrl_seqs, seq2=val_seqs))
Entropy_neg = abs(entropy(seq1=neg_ctrl_seqs_diversity, seq2=val_seqs))
Distance_pos = abs(distance(seq1=pos_ctrl_seqs, seq2=val_seqs))
Distance_neg = abs(distance(seq1=neg_ctrl_seqs_diversity, seq2=val_seqs))
Pearson_cor_pos = pearson_cor(seq1=pos_ctrl_seqs, seq2=val_seqs, random_n=1000)[1].item()

C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [6]:
## Pearson corr higher the better, all other lower the better
print('>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr' )
print(f'>>> For postivie control:\n'
      f'{MMD_pos:.5} , {Entropy_pos:.5} , {Distance_pos:.5} , {Pearson_cor_pos:.5}')
print(f'>>> For negative control:\n'
      f'{MMD_neg:.5} , {Entropy_neg:.5} , {Distance_neg:.5} , 0.000')

>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr
>>> For postivie control:
0.014261 , 2.6973e-05 , 0.009592 , nan
>>> For negative control:
0.89876 , 0.18894 , 0.50313 , 0.000


In [7]:
def save_checkpoints(discriminator, generator, optim_D, optim_G,  
               scheduler_D, scheduler_G, epoch):
    torch.save({
        'epoch': epoch,
        'disc_state_dict': discriminator.state_dict(),
        'gen_state_dict': generator.state_dict(),
        'optim_D_state_dict': optim_D.state_dict(),
        'optim_G_state_dict': optim_G.state_dict(),
        'scheduler_D_state_dict': scheduler_D.state_dict(),
        'scheduler_G_state_dict': scheduler_G.state_dict(),
    }, os.path.join('checkpoints/'+args_run_name, 'epoch_{}'.format(epoch)))
    return

def load_checkpoints(discriminator, generator, optim_D, optim_G,
                     scheduler_D, scheduler_G, epoch):
    checkpoints = torch.load(
        os.path.join('checkpoints'+args_run_name, 'epoch_{}'.format(epoch)))  
    discriminator.load_state_dict(checkpoints['disc_state_dict'])
    generator.load_state_dict(checkpoints['gen_state_dict'])
    optim_D.load_state_dict(checkpoints['optim_D_state_dict'])
    optim_G.load_state_dict(checkpoints['optim_G_state_dict'])
    scheduler_D.load_state_dict(checkpoints['scheduler_D_state_dict'])
    scheduler_G.load_state_dict(checkpoints['scheduler_G_state_dict'])
    return

## Init model

In [8]:
# init model and optimizer

Disc = Discriminator(args_dim, seq_len=seq_len, n_chars=n_chars).to(device)
Gen = Generator(args_dim, seq_len=seq_len, n_chars=n_chars, 
                z_dim=args_z_dim).to(device)

# Disc = ResNetSN_Discriminator(args_dim, seq_len, n_chars).to(device)
# Gen = ResNetSN_Generator(args_dim, seq_len, n_chars, args_z_dim).to(device)

optim_disc = optim.Adam(filter(lambda p: p.requires_grad, Disc.parameters()),
                        lr=args_lr_Disc, betas=(0.0, 0.9))
optim_gen = optim.Adam(Gen.parameters(), lr=args_lr_Gen, betas=(0.0, 0.9))
# use an exponentially decaying learning rate
scheduler_d = optim.lr_scheduler.ExponentialLR(optim_disc, gamma=0.99)
scheduler_g = optim.lr_scheduler.ExponentialLR(optim_gen, gamma=0.99)

#load_checkpoints(Disc, Gen, optim_disc, optim_gen, scheduler_d, scheduler_g, 0)
#save_checkpoints(Disc, Gen, optim_disc, optim_gen, scheduler_d, scheduler_g, 0)

In [9]:
def sample_generated(generator, epoch, run_name, n_seq=500):
    # random sampling n_seq noise, batched 100 noise at a time
    # return sampled n_seq sequences, also saving them to fasta file
    gen_seqs = []
    max_batch_size = 100
    for _ in range(n_seq//max_batch_size):
        z = torch.randn(max_batch_size, args_z_dim, device=device)
        gen_probs = generator(z)
        batched_seq = [decode_one_seq(
            gen_prob.argmax(-1).cpu().numpy()) for gen_prob in gen_probs]
        gen_seqs += batched_seq
        torch.cuda.empty_cache()
    # saving seqs in fasta
    fasta_name = 'out/' + run_name + '/seq_out/epoch_' + str(epoch) + '.fasta'
    outfile = open(fasta_name, 'w')
    for i, seq in enumerate(gen_seqs):
        seq_id = 'Epoch'+str(epoch)+'_seq'+str(i)
        outfile.write('>' + seq_id + '\n')
        outfile.write(seq + '\n')
    return gen_seqs


def write_eval_TB(writer, MMD, Entropy, Distance, Pearson, epoch):
    writer.add_scalar(args_run_name+'/MMD',  MMD, epoch)
    writer.add_scalar(args_run_name+'/abs(D_Entropy)', Entropy, epoch)
    writer.add_scalar(args_run_name+'/abs(D_Distance)', Distance, epoch)
    writer.add_scalar(args_run_name+'/Pearson Correlation', Pearson, epoch)
    return

def evaluate(epoch):
    # eval mode
    Gen.eval()
    # sample args_eval_n_seq sequences
    gen_seqs = sample_generated(Gen, epoch, args_run_name, n_seq=args_eval_n_seq)
    MMD = mmd(seq1=gen_seqs, seq2=val_seqs)
    Entropy = abs(entropy(seq1=gen_seqs, seq2=val_seqs))
    Distance = abs(distance(seq1=gen_seqs, seq2=val_seqs))
    Pearson = pearson_cor(
        seq1=gen_seqs, seq2=val_seqs, random_n=args_eval_n_seq)[1].item()
    print(f'#========== evaluating for epoch ' f'{epoch} ' f'==========#')
    print('>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr \n'
          f'{MMD:.5} , {Entropy:.5} , {Distance:.5} , {Pearson:.5}')
    # save the gap distribution plot
    plot_gap_dist(seqs=gen_seqs, run_name=args_run_name, epoch=epoch)
    write_eval_TB(TB_writer, MMD, Entropy, Distance, Pearson, epoch)
    # revert to train mode
    Gen.train()
    return


In [10]:
def train(epoch):
    for batch_idx, data in enumerate(loader):
        if data.size()[0] != args_batch_size:
            continue
        else:
            data = data.to(device)
        # train discriminator more initially
        if epoch <= args_disc_iters_init_epoch: 
            disc_iters = args_disc_iters_init
        else:
            disc_iters = args_disc_iters
        # update discriminator
        for _ in range(disc_iters):
            z = torch.randn(args_batch_size, args_z_dim, device=device)
            optim_disc.zero_grad()
            optim_gen.zero_grad()
            real_D = Disc(data)
            fake_D = Disc(Gen(z))
            if args_loss == 'hinge':
                disc_loss = nn.ReLU()(1.0 - real_D).mean() + \
                nn.ReLU()(1.0 + fake_D).mean()
            disc_loss.backward()
            optim_disc.step()
        # record output for both fake and real from discriminator
        real_prob_avg = real_D.mean()
        fake_prob_avg = fake_D.mean()
        
        # update generator
        z = torch.randn(args_batch_size, args_z_dim, device=device)
        optim_disc.zero_grad()
        optim_gen.zero_grad()
        if args_loss == 'hinge' or args_loss == 'wasserstein':
            gen_loss = -Disc(Gen(z)).mean()
        gen_loss.backward()
        optim_gen.step()
        
        # write to Tensor board
        iter = len(loader)*epoch + batch_idx
        TB_writer.add_scalars(
            args_run_name+'/GAN loss', {
            'Disc_loss': disc_loss,
            'Gen_loss': gen_loss,
        }, iter)
        TB_writer.add_scalars(
            args_run_name+'/Discriminator out', {
            'Real prob': real_prob_avg,
            'Fake prob': fake_prob_avg,
        }, iter)
        if batch_idx % 60 == 0:
            print('disc loss', disc_loss.item(), 'gen loss', gen_loss.item())
    # update lr scheduler
    scheduler_d.step()
    scheduler_g.step()
    return


In [11]:
for epoch in range(args_epoch):
    train(epoch)
    save_checkpoints(Disc, Gen, optim_disc, optim_gen, 
                     scheduler_d, scheduler_g, epoch)
    if epoch != 0:
        os.remove(os.path.join('checkpoints/'+args_run_name, 
                               'epoch_{}'.format(epoch-1)))
    # eval every 2 epoch
    if epoch % 2 == 0:
        evaluate(epoch)

disc loss 1.937320351600647 gen loss -0.032582491636276245
disc loss 0.021944429725408554 gen loss 1.0660433769226074
disc loss 0.0 gen loss 1.2910759449005127
disc loss 0.0010433066636323929 gen loss 1.280122995376587
#========== evaluating for epoch 0 ==========#
>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr 
0.77043 , 0.027427 , 1.3698 , nan
# ================== Sampling 500 sequences at epoch 0 ================== #
The loci with percentage of gap >= 95% in the cleaned MSA: 
 []
Their percentage gap are:
 []
# ====================================================== #
disc loss 0.009129540994763374 gen loss 1.4036619663238525
disc loss 0.04773637652397156 gen loss 1.2121655941009521
disc loss 0.26264524459838867 gen loss 0.9322355389595032
disc loss 0.5100820064544678 gen loss 0.8507874011993408
disc loss 0.6933207511901855 gen loss 0.896632194519043
disc loss 0.7926701307296753 gen loss 0.7051809430122375
disc loss 0.8934520483016968 gen loss 0.7078176736831665
disc loss 

C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


#========== evaluating for epoch 2 ==========#
>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr 
0.10306 , 0.0065845 , 0.0026246 , nan
# ================== Sampling 500 sequences at epoch 2 ================== #
The loci with percentage of gap >= 95% in the cleaned MSA: 
 [0, 1, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 219]
Their percentage gap are:
 [99.2, 99.6, 98.4, 98.2, 98.4, 99.2, 97.8, 99.6, 99.0, 99.4, 99.4, 99.8, 97.6, 99.0, 99.0, 99.4, 99.4, 99.8, 99.0, 98.8, 98.8, 97.4, 96.4, 97.8, 96.8, 97.8, 99.2]
# ====================================================== #
disc loss 0.9501191973686218 gen loss 0.6149370074272156
disc loss 1.0332316160202026 gen loss 0.5338668823242188
disc loss 1.1372942924499512 gen loss 0.5990921258926392
disc loss 1.3264915943145752 gen loss 0.3982865810394287
disc loss 1.0596551895141602 gen loss 0.5673372745513916
disc loss 1.1340094804763794 gen loss 0.4160841107368469
disc loss 1.11633539

C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


#========== evaluating for epoch 4 ==========#
>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr 
0.10379 , 0.0056457 , 0.0030841 , nan
# ================== Sampling 500 sequences at epoch 4 ================== #
The loci with percentage of gap >= 95% in the cleaned MSA: 
 [0, 1, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 219]
Their percentage gap are:
 [99.8, 100.0, 96.8, 98.6, 97.0, 98.0, 98.2, 98.6, 99.4, 99.2, 98.0, 98.0, 98.8, 99.2, 98.8, 98.8, 98.4, 98.0, 98.0, 98.4, 97.8, 97.2, 98.6, 99.2, 98.6, 97.4, 100.0]
# ====================================================== #
disc loss 0.9685254096984863 gen loss 0.6691887378692627
disc loss 0.7081364393234253 gen loss 0.5583799481391907
disc loss 0.8710240125656128 gen loss 0.7179151773452759
disc loss 0.6455428600311279 gen loss 0.6323131322860718
disc loss 0.6926182508468628 gen loss 0.7433863878250122
disc loss 0.6591252684593201 gen loss 0.8319277763366699
disc loss 0.600613

...

#========== evaluating for epoch 294 ==========#
>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr 
0.062583 , 0.024567 , 0.042704 , nan
# ================== Sampling 500 sequences at epoch 294 ================== #
The loci with percentage of gap >= 95% in the cleaned MSA: 
 [0, 1, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 219]
Their percentage gap are:
 [100.0, 100.0, 100.0, 100.0, 100.0, 99.8, 100.0, 99.8, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 99.6, 99.2, 100.0]
# ====================================================== #
disc loss 0.3158574402332306 gen loss 0.9772944450378418
disc loss 0.2558249533176422 gen loss 0.9958609342575073
disc loss 0.3084421753883362 gen loss 1.2004961967468262
disc loss 0.23704661428928375 gen loss 1.0179740190505981
disc loss 0.2559528946876526 gen loss 1.0296416282653809
disc loss 0.29085761308670044 gen loss 1.092417120

C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


#========== evaluating for epoch 296 ==========#
>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr 
0.058306 , 0.023253 , 0.037198 , nan
# ================== Sampling 500 sequences at epoch 296 ================== #
The loci with percentage of gap >= 95% in the cleaned MSA: 
 [0, 1, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 219]
Their percentage gap are:
 [100.0, 99.8, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 99.8, 100.0, 100.0, 99.8, 100.0, 99.6, 98.6, 100.0]
# ====================================================== #
disc loss 0.25559353828430176 gen loss 1.0628323554992676
disc loss 0.2973696291446686 gen loss 1.0441848039627075
disc loss 0.27290672063827515 gen loss 1.1617401838302612
disc loss 0.27921438217163086 gen loss 1.0279173851013184
disc loss 0.3054996430873871 gen loss 1.0720970630645752
disc loss 0.3628515601158142 gen loss 1.062458395

C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\canal\AppData\Roaming\Python\Python38\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


#========== evaluating for epoch 298 ==========#
>>> MMD | abs(D_Entropy) | abs(D_Distance) | Pearson Corr 
0.062721 , 0.024564 , 0.040826 , nan
# ================== Sampling 500 sequences at epoch 298 ================== #
The loci with percentage of gap >= 95% in the cleaned MSA: 
 [0, 1, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 219]
Their percentage gap are:
 [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 99.8, 100.0, 100.0, 100.0, 100.0, 100.0, 99.8, 99.8, 98.6, 100.0]
# ====================================================== #
disc loss 0.3295944929122925 gen loss 1.1281496286392212
disc loss 0.3692646324634552 gen loss 1.1306328773498535
disc loss 0.2926291823387146 gen loss 1.033738136291504
disc loss 0.3595472574234009 gen loss 1.1605716943740845
